In [37]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [101]:
product_type = "frozen"
num_time_periods = 7

In [102]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]


In [103]:
# constants definitions
if(product_type=="frozen"):
    warehouse_volume = 50
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 700
buffer_cost_per_m3 = 25
default_max_order = 50
max_order_exceed_multiplier = 1.5

In [104]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [105]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [106]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [107]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [108]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start=min(forecast_data['DATE']), end=max(forecast_data['DATE']), freq='D')}).astype(str)
# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

#### Heuristic

In [109]:
demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
demand = dict((date_to_index[key],value) for (key,value) in demand.items())

In [110]:
class CostMatrixCell:
    def __init__(self) -> None:
        self.orders = {}
        self.order_date = None
        self.period_end = None
        self.cost = 0
        self.avg_cost = 0
        self.extra_orders = {}

        self.clearance_cost = 0
        self.buffer_cost = 0
        self.max_order_cost = 0
        self.fixed_cost = 0
        self.per_tu_cost = 0

class Order:
    def __init__(self) -> None:
        self.item = None
        self.quantity_tu = None
        self.date = None

In [111]:
cost_distribution_buffer = 0
cost_distribution_clearance = 0
cost_distribution_max_order = 0
cost_distribution_fixed = 0
cost_distribution_per_tu = 0

In [112]:
n = len(demand.keys())
cost_matrix = [[CostMatrixCell() for __ in range(n)] for _ in range(n)]
for i in range(n):
    for j in range(n):
        if(i>j):
            cost_matrix[i][j] = None
        else:
            cost_matrix[i][j].order_date = i
            cost_matrix[i][j].period_end = j

def getOrders(cell):
    clearance_cost = 0
    for t in range(cell.order_date, cell.period_end + 1):
        for item in demand[t].keys():
            if(item not in cell.orders.keys()):
                cell.orders[item] = 0
            if(cell.order_date + shelf_life[item] < t):
                # item cannot be ordered on order_date because it would go bad by t
                if(t not in cell.extra_orders.keys()):
                    cell.extra_orders[t] = {}
                cell.extra_orders[t][item] = math.ceil(demand[t][item] / cu_per_tu[item])
                clearance_cost += ((math.ceil(demand[t][item] / cu_per_tu[item]) * cu_per_tu[item]) - demand[t][item]) * clearing_cost_per_cu[item]
            else:   
                cell.orders[item] += demand[t][item]
                # cell.orders[item] += math.ceil(demand[t][item] / cu_per_tu[item])
    for item in cell.orders.keys():
        cell.orders[item] = math.ceil(cell.orders[item]/cu_per_tu[item])
        clearance_cost += ((math.ceil(demand[t][item] / cu_per_tu[item]) * cu_per_tu[item]) - demand[t][item]) * clearing_cost_per_cu[item]

    cell.clearance_cost = clearance_cost

def fetchExtraVolume(cell, t):
    extra_volume = 0
    if(t in cell.extra_orders.keys()):
        for item in cell.extra_orders[t].keys():
            extra_volume += cell.extra_orders[t][item] * cu_per_tu[item] * volume_per_cu[item] 
    return extra_volume
            
def getCost(cell):
    getOrders(cell)
    cost = cell.clearance_cost
    volume = 0

    fixed_cost = 0
    per_tu_cost = 0
    buffer_cost = 0
    max_order_cost = 0

    for item in cell.orders.keys():
        # Fixed and per tu cost for items
        cost += (ordering_cost_per_tu[item] * cell.orders[item]) + ordering_cost_fixed[item]
        fixed_cost += ordering_cost_fixed[item]
        per_tu_cost += ordering_cost_per_tu[item] * cell.orders[item]

        # Max order penalty added
        if(cell.orders[item] > maximum_order_quantity_tu[item]):
            cost += max_order_exceed_multiplier * ordering_cost_per_tu[item] * (cell.orders[item] - maximum_order_quantity_tu[item])
            max_order_cost += max_order_exceed_multiplier * ordering_cost_per_tu[item] * (cell.orders[item] - maximum_order_quantity_tu[item])

        # Add costs for extra orders
        for extra_order_date in cell.extra_orders.keys():
            for extra_item in cell.extra_orders[extra_order_date].keys():
                cost += (ordering_cost_per_tu[extra_item] * cell.extra_orders[extra_order_date][extra_item]) + ordering_cost_fixed[extra_item]
                fixed_cost += ordering_cost_fixed[extra_item]
                per_tu_cost += ordering_cost_per_tu[extra_item] * cell.extra_orders[extra_order_date][extra_item]
                
                # Max order penalty for extra orders
                if(cell.extra_orders[extra_order_date][extra_item] > maximum_order_quantity_tu[extra_item]):
                    cost += max_order_exceed_multiplier * ordering_cost_per_tu[extra_item] * (cell.extra_orders[extra_order_date][extra_item] - maximum_order_quantity_tu[extra_item])    
                    max_order_cost += max_order_exceed_multiplier * ordering_cost_per_tu[extra_item] * (cell.extra_orders[extra_order_date][extra_item] - maximum_order_quantity_tu[extra_item])    

        # Keep track of volume
        volume += cell.orders[item] * cu_per_tu[item] * volume_per_cu[item]
    
    # Checking for extra volume and adding buffer cost
    for t in range(cell.order_date,cell.period_end+1):
        if(volume + fetchExtraVolume(cell, t) > warehouse_volume):
            cost += (volume + fetchExtraVolume(cell, t) - warehouse_volume) * buffer_cost_per_m3
            buffer_cost += (volume + fetchExtraVolume(cell, t) - warehouse_volume) * buffer_cost_per_m3

            demand_volume = 0
            for item in demand[t].keys():
                demand_volume += volume_per_cu[item] * demand[t][item]
            # Removing volume of demand sold that day
            volume -= demand_volume

            # Adding volume of extra items from that day
            volume += fetchExtraVolume(cell, t)
        else:
            break

    cell.cost = cost
    cell.avg_cost = cost / (cell.period_end - cell.order_date + 1)

    cell.buffer_cost = buffer_cost
    cell.max_order_cost = max_order_cost
    cell.fixed_cost = fixed_cost
    cell.per_tu_cost = per_tu_cost

order_date = 0
end_reached = False
obj_val = 0
schedule = {}

while(True):
    # Skipping ordering on sundays
    if(order_date%7==6):
        order_date += 1
        continue

    # Find date *until* which you want to order
    min_cost = float('inf')
    for t in range(order_date,n):
        # Skip checking demand for sundays
        if(t%7==6):
            continue

        schedule[order_date] = {}
        getCost(cost_matrix[order_date][t])

        if(cost_matrix[order_date][t].avg_cost < min_cost):
            min_cost = cost_matrix[order_date][t].avg_cost
            # If looking at ordering until last day. Make the order
            if(t==n-1):
                schedule[order_date] = cost_matrix[order_date][t].orders
                end_reached = True
                obj_val += cost_matrix[order_date][t].cost

                cost_distribution_buffer += cost_matrix[order_date][t].buffer_cost
                cost_distribution_clearance += cost_matrix[order_date][t].clearance_cost
                cost_distribution_fixed += cost_matrix[order_date][t].fixed_cost
                cost_distribution_per_tu += cost_matrix[order_date][t].per_tu_cost
                cost_distribution_max_order += cost_matrix[order_date][t].max_order_cost

                break
        # If cost increases, make order until previous day
        else:
            schedule[order_date] = cost_matrix[order_date][t-1].orders
            obj_val += cost_matrix[order_date][t-1].cost

            cost_distribution_buffer += cost_matrix[order_date][t-1].buffer_cost
            cost_distribution_clearance += cost_matrix[order_date][t-1].clearance_cost
            cost_distribution_fixed += cost_matrix[order_date][t-1].fixed_cost
            cost_distribution_per_tu += cost_matrix[order_date][t-1].per_tu_cost
            cost_distribution_max_order += cost_matrix[order_date][t-1].max_order_cost

            # Place next order starting from today
            order_date = t
            break
    
    if(order_date >= n or end_reached):
        break

print(cost_distribution_buffer)
print(cost_distribution_clearance)
print(cost_distribution_fixed)
print(cost_distribution_per_tu)
print(cost_distribution_max_order)
obj_val

441.5866153999996
1797.1999999999996
4131.8
59330.299999999974
9.3


65710.18661539997

In [113]:
result_dict = {}

for day, articles in schedule.items():
    for article, quantity in articles.items():
        if article in result_dict:
            result_dict[article][day] = quantity
        else:
            result_dict[article] = {day: quantity}


In [114]:
schedule = result_dict
max_improvement = math.inf
cost = obj_val
modified_schedule = {}

count = 0
while count <=10:
    for item in schedule:
        buffer_penalty = 0
        max_order_penalty = 0 
        cost_improvement = 0
        modified_orders = {}
        # Check if the item has a second order date
        if len(schedule[item]) > 1:
            order_days = list(schedule[item].keys())
            for i in range (0,len(order_days)-1, 2):
                day1 = i
                day2 = i +1
                    
                if order_days[day2] - order_days[day1] <= shelf_life[item]:
                    tot_orders = schedule[item][order_days[day1]]+ schedule[item][order_days[day2]]
                    vol = tot_orders * cu_per_tu[item] * volume_per_cu[item]
                    if vol > warehouse_volume:
                        buffer_penalty += cost + buffer_cost_per_m3 * (vol - warehouse_volume)
                    if tot_orders > maximum_order_quantity_tu[item]:
                        max_order_penalty += ( tot_orders - maximum_order_quantity_tu[item]) * max_order_exceed_multiplier * ordering_cost_per_tu[item]
                    delta = - ordering_cost_fixed[item] + buffer_penalty + max_order_penalty 
                    if delta < 0:
                        cost += delta
                        modified_orders[day1] = tot_orders
                    else:
                        modified_orders[day1] = schedule[item][order_days[day1]]
                        modified_orders[day2] = schedule[item][order_days[day2]]
            modified_schedule[item] = modified_orders
        
        else: 
            modified_schedule[item] = schedule[item]


        if cost!=0 and cost < max_improvement:
            max_improvement = cost
    
    schedule = modified_schedule
    obj_val = max_improvement
    count +=1

    print(max_improvement)

65710.18661539997
65710.18661539997
65710.18661539997
65710.18661539997
65710.18661539997
65710.18661539997
65710.18661539997
65710.18661539997
65710.18661539997
65710.18661539997
65710.18661539997
